In [3]:
import json
import os
import requests

from tensorflow import keras
import pandas as pd

%load_ext lab_black

In [43]:
def get_data_table(rows: int):
    import pandas as pd
    from trino.dbapi import Connection

    with Connection(
        host="trino.trino",
        port="8080",
        user="anybody",
        catalog="jtopen",
        schema="demo",
    ) as conn:
        link = conn.cursor()
        link.execute(f"SELECT * FROM fraud LIMIT {rows}")
        return pd.DataFrame(link.fetchall(), columns=[i.name for i in link.description])


rdf = get_data_table(100000)
print(f"Retrieved {len(rdf)} rows")
rdf.head()

Retrieved 100000 rows


,user,card,year,month,day,time,amount,use chip,merchant name,merchant city,merchant state,zip,mcc,errors?,is fraud?
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750,5300,...,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754,5411,...,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754,5411,...,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754,5651,...,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750,5912,...,No


In [44]:
rdf["is fraud?"].value_counts()

No     99874
Yes      126
Name: is fraud?, dtype: int64

In [45]:
test_form = {
    "user": 0,
    "card": 0,
    "merchant name": "Stop n Shop",
    "amount": 0,
    "year": 2015,
    "month": 1,
    "day": 1,
    "transaction type": "payment method",
    "merchant city": "Bucyrus",
    "merchant state": "OH",
    "zip": 0,
    "errors": "None",
    "mcc": 0,
}

test_input_df = pd.DataFrame([test_form])
test_input_df

,user,card,merchant name,amount,year,month,day,transaction type,merchant city,merchant state,zip,errors,mcc
0,0,0,Stop n Shop,0,2015,1,1,payment method,Bucyrus,OH,0,None,0


In [11]:
test_input_df.dtypes

user                 int64
card                 int64
merchant name       object
amount               int64
year                 int64
month                int64
day                  int64
transaction type    object
merchant city       object
merchant state      object
zip                  int64
errors              object
mcc                  int64
dtype: object

In [48]:
data_sample = rdf.sample(3, random_state=123).reset_index()
data_sample["merchant name"]

0    -5475680618560174533
1     1839636615252322303
2     1902624851238409582
Name: merchant name, dtype: object

In [54]:
from decimal import Decimal


def convert_df_columns(data_sample, type=int):
    for col in data_sample.columns:
        if isinstance(data_sample[col].iloc[0], Decimal):
            data_sample[col] = data_sample[col].apply(int)
    return data_sample


d = convert_df_columns(data_sample).to_dict(orient="records")
d
# with open('thisistestdata.json', 'w') as f:
#     json.dump(d, f)

[{'index': 42083,
  'user': 2,
  'card': 1,
  'year': 2005,
  'month': 2,
  'day': 15,
  'time': '06:33',
  'amount': '$13.64   ',
  'use chip': 'Swipe Transaction ',
  'merchant name': -5475680618560174533,
  'merchant city': 'Spring Valley             ',
  'merchant state': 'CA                              ',
  'zip': 91977,
  'mcc': 5942,
  'errors?': '                                                    ',
  'is fraud?': 'No '},
 {'index': 71825,
  'user': 3,
  'card': 0,
  'year': 2013,
  'month': 2,
  'day': 27,
  'time': '16:47',
  'amount': '$3039.73 ',
  'use chip': 'Swipe Transaction ',
  'merchant name': 1839636615252322303,
  'merchant city': 'Brooklyn                  ',
  'merchant state': 'NY                              ',
  'zip': 11208,
  'mcc': 3000,
  'errors?': '                                                    ',
  'is fraud?': 'No '},
 {'index': 99535,
  'user': 5,
  'card': 0,
  'year': 2002,
  'month': 1,
  'day': 5,
  'time': '17:42',
  'amount': '$69.41   ',

In [57]:
fraud_sample = rdf[rdf["is fraud?"] == "Yes"]
# fraud_sample.columns = [column.replace(" ", "_") for column in fraud_sample.columns]
fs = fraud_sample.sample(2, random_state=1234)

convert_df_columns(fs).to_dict(orient="records")

[{'user': 2,
  'card': 4,
  'year': 2015,
  'month': 9,
  'day': 2,
  'time': '14:09',
  'amount': '$149.84  ',
  'use chip': 'Online Transaction',
  'merchant name': 3452760747765970571,
  'merchant city': ' ONLINE                   ',
  'merchant state': '                                ',
  'zip': 0,
  'mcc': 3174,
  'errors?': '                                                    ',
  'is fraud?': 'Yes'},
 {'user': 2,
  'card': 1,
  'year': 2006,
  'month': 5,
  'day': 25,
  'time': '16:17',
  'amount': '$37.77   ',
  'use chip': 'Swipe Transaction ',
  'merchant name': 3987158270252316808,
  'merchant city': 'Algiers                   ',
  'merchant state': 'Algeria                         ',
  'zip': 0,
  'mcc': 5921,
  'errors?': '                                                    ',
  'is fraud?': 'Yes'}]

In [35]:
fraud_sample.iloc[0].to_dict()

{'user': Decimal('0'),
 'card': Decimal('0'),
 'year': Decimal('2015'),
 'month': Decimal('11'),
 'day': Decimal('15'),
 'time': '12:55',
 'amount': '$287.13  ',
 'use_chip': 'Online Transaction',
 'merchant_name': Decimal('-8194607650924472520'),
 'merchant_city': ' ONLINE                   ',
 'merchant_state': '                                ',
 'zip': Decimal('0'),
 'mcc': Decimal('3001'),
 'errors?': '                                                    ',
 'is_fraud?': 'Yes'}

In [15]:
fraud_sample["merchant_name"].value_counts()

-4282466774399734331    52
1913477460590765860     51
9057735476014445185     35
3189517333335617109     29
-2916542501422915698    24
                        ..
5927537557379097933      1
-4117079033496807984     1
1054351502284293051      1
-289096924892923056      1
985550334332641081       1
Name: merchant_name, Length: 294, dtype: int64

In [17]:
fraud_sample.loc[fraud_sample["merchant_name"] == -4282466774399734331]

,user,card,year,month,day,time,amount,use_chip,merchant_name,merchant_city,merchant_state,zip,mcc,errors?,is_fraud?
66598,2,4,2013,10,11,06:55,$98.85,Online Transaction,-4282466774399734331,ONLINE,,0,4829,...,Yes
66599,2,4,2013,10,11,07:21,$142.82,Online Transaction,-4282466774399734331,ONLINE,,0,4829,...,Yes
66600,2,4,2013,10,11,09:32,$96.38,Online Transaction,-4282466774399734331,ONLINE,,0,4829,...,Yes
67982,2,4,2015,9,1,11:05,$78.16,Online Transaction,-4282466774399734331,ONLINE,,0,4829,...,Yes
180976,9,0,2014,4,6,14:36,$10.75,Online Transaction,-4282466774399734331,ONLINE,,0,4829,...,Yes
194200,13,0,2016,7,24,10:41,$267.66,Online Transaction,-4282466774399734331,ONLINE,,0,4829,...,Yes
194201,13,0,2016,7,24,14:47,$57.87,Online Transaction,-4282466774399734331,ONLINE,,0,4829,...,Yes
197725,13,1,2015,8,3,10:44,$140.77,Online Transaction,-4282466774399734331,ONLINE,,0,4829,...,Yes
197726,13,1,2015,8,3,11:00,$82.37,Online Transaction,-4282466774399734331,ONLINE,,0,4829,...,Yes
218231,15,4,2008,3,2,16:24,$19.79,Online Transaction,-4282466774399734331,ONLINE,,0,4829,...,Yes


In [20]:
print(rdf["year"].min())
print(rdf["year"].max())

1996
2020


In [22]:
fraud_sample["use_chip"].value_counts()

Online Transaction    518
Swipe Transaction     294
Chip Transaction      180
Name: use_chip, dtype: int64

In [29]:
fraud_sample["merchant_state"].unique()

array(['                                ',
       'CA                              ',
       'NY                              ',
       'Italy                           ',
       'Haiti                           ',
       'Algeria                         ',
       'OH                              ',
       'MI                              ',
       'OK                              ',
       'TX                              ',
       'NE                              ',
       'CO                              ',
       'AZ                              ',
       'IN                              ',
       'Nigeria                         ',
       'Mexico                          ',
       'Tuvalu                          ',
       'TN                              ',
       'FL                              ',
       'MA                              ',
       'KS                              ',
       'GA                              ',
       'NJ                              ',
       'SC 